## IMPORT LIB

In [1]:
import pandas as pd 
import numpy as np

## TOY DATA

In [2]:
toyDataMatriks = [
    [3,	0,	0,	0,	0,	0,	2],
    [1,	2,	4,	2,	2,	0,	0],
    [0,	4,	1,	0,	0,	2,	4],
    [0,	0,	0,	0,	3,	2,	4]
]

In [3]:
# toydata to dataframe

dfToyData = pd.DataFrame(toyDataMatriks, columns=['item-1', 'item-2', 'item-3', 'item-4','item-5','item-6', 'item-7'], index=['user-1', 'user-2', 'user-3', 'user-4'])
dfToyData

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,0,0,0,0,0,2
user-2,1,2,4,2,2,0,0
user-3,0,4,1,0,0,2,4
user-4,0,0,0,0,3,2,4


In [4]:
dfToyData2 = pd.DataFrame(
    toyDataMatriks,
    columns=[1, 2, 3, 4, 5, 6, 7],  # kolom menggunakan integer
    index=[1, 2, 3, 4]  # index juga menggunakan integer
)

dfToyData2

,1,2,3,4,5,6,7
1,3,0,0,0,0,0,2
2,1,2,4,2,2,0,0
3,0,4,1,0,0,2,4
4,0,0,0,0,3,2,4


## MEAN RATING

In [5]:
def meanRating(rating):
    num = rating.sum(axis=1)
    den = np.count_nonzero(rating, axis=1)
    # menghindari pembagian dengan 0
    calCulatedMean  = np.where(den != 0, num / den, 0)
    # menghitung mean
    meanRatingUser = pd.DataFrame(calCulatedMean, index=rating.index, columns=['meanRating'])

    return meanRatingUser

In [6]:
cal_MeanRatingUser = meanRating(dfToyData)
cal_MeanRatingUser

,meanRating
user-1,2.50
user-2,2.20
user-3,2.75
user-4,3.00


In [7]:
cal_MeanRatingItem = meanRating(dfToyData.T)
cal_MeanRatingItem

,meanRating
item-1,2.000000
item-2,3.000000
item-3,2.500000
item-4,2.000000
item-5,2.500000
item-6,2.000000
item-7,3.333333


## MEAN CENTERD

In [8]:
def meanCenterd(rating, meanRating):
    npRatingMatriks = np.array(rating)
    npMeanRatingUser = np.array(meanRating)
    # !0
    meanCenteredUser = np.where(npRatingMatriks!=0, npRatingMatriks - npMeanRatingUser,0)
    dfMeanCenteredUser = pd.DataFrame(meanCenteredUser, index=rating.index, columns=rating.columns)

    return dfMeanCenteredUser

In [9]:
cal_MeanCenteredUser = meanCenterd(dfToyData, cal_MeanRatingUser)
cal_MeanCenteredUser

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,0.5,0.00,0.00,0.0,0.0,0.00,-0.50
user-2,-1.2,-0.20,1.80,-0.2,-0.2,0.00,0.00
user-3,0.0,1.25,-1.75,0.0,0.0,-0.75,1.25
user-4,0.0,0.00,0.00,0.0,0.0,-1.00,1.00


In [10]:
cal_MeanCenteredItem = meanCenterd(dfToyData.T, cal_MeanRatingItem)
cal_MeanCenteredItem


,user-1,user-2,user-3,user-4
item-1,1.000000,-1.0,0.000000,0.000000
item-2,0.000000,-1.0,1.000000,0.000000
item-3,0.000000,1.5,-1.500000,0.000000
item-4,0.000000,0.0,0.000000,0.000000
item-5,0.000000,-0.5,0.000000,0.500000
item-6,0.000000,0.0,0.000000,0.000000
item-7,-1.333333,0.0,0.666667,0.666667


In [11]:
cal_MeanCenteredItemTranpose = cal_MeanCenteredItem.T
cal_MeanCenteredItemTranpose

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,1.0,0.0,0.0,0.0,0.0,0.0,-1.333333
user-2,-1.0,-1.0,1.5,0.0,-0.5,0.0,0.000000
user-3,0.0,1.0,-1.5,0.0,0.0,0.0,0.666667
user-4,0.0,0.0,0.0,0.0,0.5,0.0,0.666667


## SIMILARITY UB DAN IB

In [12]:
# iPenggunaU = len(itemRatedPenggunaU)
# iPenggunaV = len(itemRatedPenggunav)

In [13]:
def SimilarityRJ(RatingMatriks, index1, index2, jenis="user-based"):
    # jika jenis == "user-based":
    if jenis == "user-based":
        Rated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] != 0])
        # print("itemRated1: ", itemRated1)
        Rated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] != 0])
        NotRated1 = set(RatingMatriks.columns[RatingMatriks.iloc[index1, :] == 0])
        NotRated2 = set(RatingMatriks.columns[RatingMatriks.iloc[index2, :] == 0])
        # print("Rated2: ", Rated2)
    elif jenis == "item-based":
        # mencari index user yang telah memberi rating pada item1 dan item2
        Rated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] != 0])
        Rated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] != 0])
        NotRated1 = set(RatingMatriks.index[RatingMatriks.iloc[:, index1] == 0])
        NotRated2 = set(RatingMatriks.index[RatingMatriks.iloc[:, index2] == 0])
    # menghitung banyaknya item yang telah diberi rating oleh user1 dan user2
    intersection = len(Rated1.intersection(Rated2))
    #menghitung jumlah total item yang telah diberi rating oleh user dan jumlah total item yang telah diberi rating oleh user
    CountNotRated1 = len(NotRated1)
    CountNotRated2 = len(NotRated2)
    # menghitung similarity
    if intersection != 0:
        rumusRelevantJaccard = 1/(1+(1/intersection)+(CountNotRated1/(1+CountNotRated1))+(1/(1+CountNotRated2)))
    elif intersection == 0:
        rumusRelevantJaccard = 0
    return rumusRelevantJaccard

In [14]:
calSimilarityRJ = SimilarityRJ(dfToyData2, 6, 6, jenis="item-based")
print("Similarity Jaccard: ", calSimilarityRJ)

Similarity Jaccard:  0.4285714285714286


In [15]:
def simRJUser(rating, penggunaU, penggunaV):
    # mendapatkan item yang diberi rating oleh penggunaU dan penggunaV
    # itemRatedPenggunaU2 = set(np.where(rating.loc[penggunaU, :] != 0)[0])
    # print(itemRatedPenggunaU2)
    itemRatedPenggunaU = set(rating.columns[rating.iloc[penggunaU, : ] != 0])
    print(itemRatedPenggunaU)
    # print(itemRatedPenggunaU)
    itemRatedPenggunav = set(rating.columns[rating.iloc[penggunaV, :] != 0])
    # print(itemRatedPenggunav)
    # mencari irisan dari item yang diberi rating oleh penggunaU dan penggunaV
    intersection = len(itemRatedPenggunaU.intersection(itemRatedPenggunav))
    # print(intersection)
    # komplemen penggunaU dan penggunaV
    itemNotRatedPenggunaU = len(set(rating.columns[rating.iloc[penggunaU, :] == 0]))
    itemNotRatedPenggunav = len(set(rating.columns[rating.iloc[penggunaV, :] == 0]))

    # menghitung similarity
    if intersection != 0:
        rumusSimRJUser = 1/(1+(1/intersection)+(itemNotRatedPenggunaU/(1+itemNotRatedPenggunaU))+(1/(1+itemNotRatedPenggunav)))
    else:
        rumusSimRJUser = 0
    
    return rumusSimRJUser

In [16]:
# menghitung similarity antar pengguna
cal_simRJUser = simRJUser(dfToyData2,0, 0)
print(f"similarity user-1 dan user-2 = {cal_simRJUser}")

{1, 7}
similarity user-1 dan user-2 = 0.4


In [17]:
def simRJItem(rating, itemU, itemV):
    # mendapatkan pengguna yang memberi rating pada itemU dan itemV
    userRatedItemU = set(rating.index[rating.iloc[:, itemU] != 0])
    userRatedItemV = set(rating.index[rating.iloc[:, itemV] != 0])
    
    # mencari irisan dari pengguna yang memberi rating pada itemU dan itemV
    intersection = len(userRatedItemU.intersection(userRatedItemV))

    # komplemen user 
    userNotRatedItemU = len(set(rating.index[rating.iloc[:, itemU] == 0]))
    userNotRatedItemV = len(set(rating.index[rating.iloc[:, itemV] == 0]))
    # menghitung similarity
    if intersection != 0:
        rumusSimRJItem = 1/(1+(1/intersection)+(userNotRatedItemU/(1+userNotRatedItemU))+(1/(1+userNotRatedItemV)))
    elif intersection == 0:
        rumusSimRJItem = 0
    return rumusSimRJItem

In [18]:
calR = simRJItem(dfToyData, 0,1)
print(f"similarity item-3 dan item-3 = {calR}")

similarity item-3 dan item-3 = 0.3333333333333333


### KOMBINASI USER ATAU ITEM

In [19]:
def similarityAllUser2(rating, jenis='user'):
    if jenis == 'user':
        jumlah = rating.shape[0]
        indeks = rating.index
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = simRJUser(rating, i, j)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=indeks, columns=indeks)
    elif jenis == 'item':
        jumlah = rating.shape[1]
        kolom = rating.columns
        simMatriks = np.zeros((jumlah, jumlah))
        for i in range(jumlah):
            for j in range(i, jumlah):
                nilaiSim = simRJItem(rating, i, j)
                simMatriks[i][j] = nilaiSim
                simMatriks[j][i] = nilaiSim
        simMatriksDf = pd.DataFrame(simMatriks, index=kolom, columns=kolom)
    return simMatriksDf

In [20]:
cal_simAllUser = similarityAllUser2(dfToyData, 'user')
cal_simAllUser

{'item-7', 'item-1'}
{'item-7', 'item-1'}
{'item-7', 'item-1'}
{'item-7', 'item-1'}
{'item-4', 'item-3', 'item-1', 'item-2', 'item-5'}
{'item-4', 'item-3', 'item-1', 'item-2', 'item-5'}
{'item-4', 'item-3', 'item-1', 'item-2', 'item-5'}
{'item-7', 'item-6', 'item-2', 'item-3'}
{'item-7', 'item-6', 'item-2', 'item-3'}
{'item-7', 'item-5', 'item-6'}


,user-1,user-2,user-3,user-4
user-1,0.400000,0.315789,0.324324,0.329670
user-2,0.315789,0.454545,0.413793,0.348837
user-3,0.324324,0.413793,0.444444,0.408163
user-4,0.329670,0.348837,0.408163,0.428571


In [21]:
cal_simAllItem = similarityAllUser2(dfToyData, 'item')
cal_simAllItem

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
item-1,0.400000,0.333333,0.333333,0.342857,0.333333,0.000000,0.315789
item-2,0.333333,0.400000,0.400000,0.342857,0.333333,0.333333,0.315789
item-3,0.333333,0.400000,0.400000,0.342857,0.333333,0.333333,0.315789
item-4,0.342857,0.342857,0.342857,0.333333,0.324324,0.000000,0.000000
item-5,0.333333,0.333333,0.333333,0.324324,0.400000,0.333333,0.315789
item-6,0.000000,0.333333,0.333333,0.000000,0.333333,0.400000,0.375000
item-7,0.315789,0.315789,0.315789,0.000000,0.315789,0.375000,0.428571


## TOPK

In [22]:
def topKNeigbors(RatingMatriks, k=2):

    # np array
    ## aturan 1 : mengambil similarity dari matriks rating tertinggi
    ## aturan 2 : abaikan jika dengan dirinya sendiri jadi ambil index lain
    similarityFunction = np.array(cal_simAllUser(RatingMatriks))
    # abaikan diagonal dan ambil index lain
    selfNotIncluded = np.where(np.eye(similarityFunction.shape[0])==1, 0, similarityFunction)
    # ambil index dari similarity tertinggi
    topK = np.argsort(selfNotIncluded, axis=1)[:, -k:]
    return topK

In [23]:
def TopKTetangga(similarity, k):
    return np.argsort(-similarity)[:k]  # Mengambil k tetangga terdekat


In [24]:
def TetanggaK(target_user, target_item, matriks_rating, similarity, k, jenis="user"):
    if jenis == "user":
        # Cari user lain (index) yang memberi rating pada item tertentu
        filter_indices = [i for i in range(matriks_rating.shape[0])
                          if i != target_user and matriks_rating.iloc[i, target_item] > 0]

        sim_scores = similarity.iloc[target_user, filter_indices].values
    else:
        # Cari item lain (index) yang pernah dinilai oleh user tertentu
        filter_indices = [j for j in range(matriks_rating.shape[1])
                          if j != target_user and matriks_rating.iloc[target_item, j] > 0]

        sim_scores = similarity.iloc[target_user, filter_indices].values

    # Urutkan index berdasarkan similarity
    top_k_idx = np.argsort(-sim_scores)[:k]
    top_k_indices = [filter_indices[i] for i in top_k_idx]

    return top_k_indices

## yang digunakan

In [25]:
def TetanggaK(target_user, target_item, rating_matrix_np, similarity_np, k, jenis="user-based"):
    """
    Mengembalikan indeks K tetangga terdekat (user atau item) berdasarkan similarity.
    """
    rating_matrix = rating_matrix_np
    if jenis == "user-based":
        # Ambil semua rating terhadap item target, data rating untuk item target
        item_ratings = rating_matrix[:, target_item]
        tetangga_idx = np.where((item_ratings != 0))[0]
        sim_scores = similarity_np[target_user, tetangga_idx]
        # cek jumalh rating
        martiks = rating_matrix[target_user, tetangga_idx]
        # print(f"matriks rating {martiks.shape}")

    else:  # item-based
        # Di item-based, user tetap user, item adalah target
        user_ratings = rating_matrix_np[target_user, :]
        tetangga_idx = np.where((user_ratings != 0))[0]
        # Ambil skor similarity item-item
        sim_scores = similarity_np[tetangga_idx, target_item]
    # Urutkan tetangga berdasarkan similarity tertinggi
    if len(sim_scores) == 0:
        return []

    # print(f"sim scores {sim_scores}")
    # print(f"sim scores {sim_scores.shape}")
    sorted_idx = np.argsort(-sim_scores)[:k]
    # print(f"sorted idx {sorted_idx}")
    # print(f"sim from soreted {sorted_idx} {sim_scores[sorted_idx]}")
    return tetangga_idx[sorted_idx]



## PREDIKSI

In [26]:
def prediksiCF_v2(RatingMatriks, similarityFunction, mean, meanCen, user="user-2", item="item-2", k=2, jenis="userBased"):
    if jenis == "userBased":
        indeks_user = RatingMatriks.index.get_loc(user)
        indeks_item = RatingMatriks.columns.get_loc(item)
        # Ambil top-k tetangga user yang juga memberi rating ke item
        tetangga = TetanggaK(
            target_user=indeks_user,
            target_item=indeks_item,
            matriks_rating=RatingMatriks,
            similarity=similarityFunction,
            k=k,
            jenis="user"
        )
        print("TetanggaK", tetangga)
        # Hitung pembilang dan penyebut
        pembilang = 0
        penyebut = 0
        for tetangga_idx in tetangga:
            nama_tetangga = RatingMatriks.index[tetangga_idx]
            nilai_meanCen = meanCen.loc[nama_tetangga, item]
            # print(f"Mean centered rating for {nama_tetangga} on {item}: {nilai_meanCen}")
            nilai_sim = similarityFunction.loc[user, nama_tetangga]
            # print(f"Similarity between {user} and {nama_tetangga}: {nilai_sim}")

            pembilang += nilai_meanCen * nilai_sim
            penyebut += abs(nilai_sim)

        if penyebut != 0:
            prediksi = mean.loc[user] + (pembilang / penyebut)
            print("mean", mean.loc[user])
        else:
            prediksi = mean.loc[user]
    elif jenis == "itemBased":
        indeks_item = RatingMatriks.columns.get_loc(item)
        indeks_user = RatingMatriks.index.get_loc(user)
        # Ambil top-k tetangga item yang juga dinilai oleh user
        tetangga = TetanggaK(
            target_user=indeks_item,
            target_item=indeks_user,
            matriks_rating=RatingMatriks,
            similarity=similarityFunction,
            k=k,
            jenis="item"
        )
        # Hitung pembilang dan penyebut
        pembilang = 0
        penyebut = 0
        for tetangga_idx in tetangga:
            nama_tetangga = RatingMatriks.columns[tetangga_idx]
            nilai_meanCen = meanCen.loc[user, nama_tetangga]
            print("nilai mean cen", nilai_meanCen)
            nilai_sim = similarityFunction.loc[nama_tetangga, item]
            print("nilai sim", nilai_sim)
            pembilang += nilai_meanCen * nilai_sim
            print("pembilnag 1", pembilang)
            penyebut += abs(nilai_sim)
        if penyebut != 0:
            prediksi = mean.loc[item] + (pembilang / penyebut)
            print("mean", mean.loc[item])
            print("pembilang", pembilang, "penyebut", penyebut)
        else:
            prediksi = mean.loc[item]
    return [item, float(prediksi)]

## yang digunakan 

In [27]:
def PrediksiCF(RatingMatriks, similarityFunction, mean, meanCen, user, item, k=2, jenis="user-based"):
    """
    Menghitung prediksi rating menggunakan Collaborative Filtering berbasis user/item, versi optimal.
    """

    # Cache ke bentuk NumPy
    rating_np = RatingMatriks.to_numpy()

    # cek jumlah yang pernah memberikan rating 
    # print(f"jumlah rating item {item} {np.count_nonzero(rating_np[:, item])}")


    similarity_np = similarityFunction.to_numpy()
    # print(similarity_np.shape)
    mean_np = mean.to_numpy()
    # print(mean_np.shape)
    meanCen_np = meanCen.to_numpy()
    print("mean cen", meanCen_np.shape)

    # Set indeks untuk pencarian tetangga
    target_user = user
    target_item = item
    print(f"target user {target_user} {target_item}")
    mean_value = mean_np[user] if jenis == "user-based" else mean_np[item]
    print(f"mean {mean_value}")

    # Cari tetangga
    tetangga = TetanggaK(target_user, target_item, rating_np, similarity_np, k, jenis)
    # print(f"tetangga {tetangga}")
    if len(tetangga) == 0:
        return float(mean_value)

    # Ambil similarity dan mean centered rating
    if jenis == "user-based":
        sim = similarity_np[user, tetangga] # ambil similarity user-user
        print(f'sim {sim}')
        mean_cen = meanCen_np[tetangga, item] 
        print(f"mean cen {mean_cen}")
        # cek tetangga dan item pada mean cen
        print(f"mean cen {meanCen_np[tetangga, item].shape}")
    else:  # item-based
        sim = similarity_np[tetangga, item] # ambil similarity item-item
        # print(f"sim item {sim.shape}") 
        mean_cen = meanCen_np[user, tetangga] # ambil mean centered rating
        print(f'sim {sim}')
        print(f"mean cen {mean_cen}")
    # vektor ((a1 * b1) + (a2 * b2))
    pembilang = np.dot(mean_cen, sim)
    print(f"pembilang {pembilang}")
    penyebut = np.sum(np.abs(sim))
    print(f"penyebut {penyebut}")

    prediksi = mean_value + (pembilang / penyebut) if penyebut != 0 else mean_value
    print(f"prediksi {mean_value} + {pembilang} / {penyebut}")
    return float(prediksi)

In [43]:
cal_PrediksiRJUser = PrediksiCF(dfToyData2, cal_simAllUser, cal_MeanRatingUser, cal_MeanCenteredUser, user=0, item=2, k=2, jenis="user-based")
print(f"prediksi rating user-2 untuk item-2 = {cal_PrediksiRJUser}")

mean cen (4, 7)
target user 0 2
mean [2.5]
sim [0.32432432 0.31578947]
mean cen [-1.75  1.8 ]
mean cen (2,)
pembilang 0.0008534850640112834
penyebut 0.6401137980085347
prediksi [2.5] + 0.0008534850640112834 / 0.6401137980085347
prediksi rating user-2 untuk item-2 = 2.501333333333333


In [29]:
cal_PrediksiRJItem = PrediksiCF(dfToyData2, cal_simAllItem, cal_MeanRatingItem, cal_MeanCenteredItemTranpose, user=0, item=2, k=2, jenis="item-based")
print(f"prediksi rating user-1 untuk item-2 = {cal_PrediksiRJItem}")


mean cen (4, 7)
target user 0 2
mean [2.5]
sim [0.33333333 0.31578947]
mean cen [ 1.         -1.33333333]
pembilang -0.08771929824561414
penyebut 0.6491228070175439
prediksi [2.5] + -0.08771929824561414 / 0.6491228070175439
prediksi rating user-1 untuk item-2 = 2.364864864864865


## PREDIKSI KESELURUHAN

In [30]:
def prediksi_semua_matriks2(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
    """
    Mengisi semua rating kosong dalam matriks dengan hasil prediksi CF.
    """
    # salin rating matriks
    hasil_matriks = RatingMatriks.copy()
    # mendapatkan ukuran matriks rating/ ambil
    num_users, num_items = hasil_matriks.shape
    # loop setiap user
    for user in range(num_users):
        # loop setiap item
        for item in range(num_items):
            if hasil_matriks.iloc[user, item] == 0:
                pred_rating = PrediksiCF(
                    RatingMatriks=RatingMatriks,
                    similarityFunction=similarityFunction,
                    mean=mean,
                    meanCen=meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis # user-based atau item-based
                )
                # isi prediksi hasil ke dalam matriks
                hasil_matriks.iloc[user, item] = pred_rating
    return hasil_matriks

In [31]:
# def hitungKeseluruhanPrediksi(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="userBased"):
#     # Membuat DataFrame kosong untuk menyimpan prediksi
#     prediksiMatriks = pd.DataFrame(index=RatingMatriks.index, columns=RatingMatriks.columns)
    
#     # Iterasi untuk setiap user dan item dalam matriks rating
#     for user in RatingMatriks.index:
#         for item in RatingMatriks.columns:
#             # Cek apakah rating untuk (user, item) adalah 0 atau tidak ada rating
#             if RatingMatriks.loc[user, item] == 0:
#                 # Jika tidak ada rating, prediksi nilai rating menggunakan Collaborative Filtering
#                 prediksiMatriks.loc[user, item] = prediksiCF_v2(
#                     RatingMatriks,
#                     similarityFunction,
#                     mean,
#                     meanCen,
#                     user=user,
#                     item=item,
#                     k=k,
#                     jenis=jenis
#                 )[1]
#             else:
#                 # Jika sudah ada rating, simpan rating asli dari RatingMatriks
#                 prediksiMatriks.loc[user, item] = RatingMatriks.loc[user, item]

#     return prediksiMatriks


In [32]:
cal_HitungAllPrediksiUserRJ = prediksi_semua_matriks2(dfToyData2, cal_simAllUser, cal_MeanRatingUser, cal_MeanCenteredUser, k=2, jenis="user-based")
cal_HitungAllPrediksiUserRJ

mean cen (4, 7)
target user 0 1
mean [2.5]
sim [0.32432432 0.31578947]
mean cen [ 1.25 -0.2 ]
mean cen (2,)
pembilang 0.3422475106685632
penyebut 0.6401137980085347
prediksi [2.5] + 0.3422475106685632 / 0.6401137980085347
mean cen (4, 7)
target user 0 2
mean [2.5]
sim [0.32432432 0.31578947]
mean cen [-1.75  1.8 ]
mean cen (2,)
pembilang 0.0008534850640112834
penyebut 0.6401137980085347
prediksi [2.5] + 0.0008534850640112834 / 0.6401137980085347
mean cen (4, 7)
target user 0 3
mean [2.5]
sim [0.31578947]
mean cen [-0.2]
mean cen (1,)
pembilang -0.06315789473684216
penyebut 0.3157894736842105
prediksi [2.5] + -0.06315789473684216 / 0.3157894736842105
mean cen (4, 7)
target user 0 4
mean [2.5]
sim [0.32967033 0.31578947]
mean cen [ 0.  -0.2]
mean cen (2,)
pembilang -0.06315789473684216
penyebut 0.6454598033545401
prediksi [2.5] + -0.06315789473684216 / 0.6454598033545401
mean cen (4, 7)
target user 0 5
mean [2.5]
sim [0.32967033 0.32432432]
mean cen [-1.   -0.75]
mean cen (2,)
pembilang 

,1,2,3,4,5,6,7
1,3.000000,3.034667,2.501333,2.30,2.402151,1.623978,2.000000
2,1.000000,2.000000,4.000000,2.00,2.000000,1.335647,3.335647
3,2.296970,4.000000,1.000000,2.55,2.649315,2.000000,4.000000
4,2.625989,3.581818,2.885893,2.80,3.000000,2.000000,4.000000


In [33]:
cal_HitungAllPrediksiItemRJ =prediksi_semua_matriks2(dfToyData2, cal_simAllItem, cal_MeanRatingItem, cal_MeanCenteredItemTranpose, k=2, jenis="item-based")
cal_HitungAllPrediksiItemRJ

mean cen (4, 7)
target user 0 1
mean [3.]
sim [0.33333333 0.31578947]
mean cen [ 1.         -1.33333333]
pembilang -0.08771929824561414
penyebut 0.6491228070175439
prediksi [3.] + -0.08771929824561414 / 0.6491228070175439
mean cen (4, 7)
target user 0 2
mean [2.5]
sim [0.33333333 0.31578947]
mean cen [ 1.         -1.33333333]
pembilang -0.08771929824561414
penyebut 0.6491228070175439
prediksi [2.5] + -0.08771929824561414 / 0.6491228070175439
mean cen (4, 7)
target user 0 3
mean [2.]
sim [0.34285714 0.        ]
mean cen [ 1.         -1.33333333]
pembilang 0.34285714285714286
penyebut 0.34285714285714286
prediksi [2.] + 0.34285714285714286 / 0.34285714285714286
mean cen (4, 7)
target user 0 4
mean [2.5]
sim [0.33333333 0.31578947]
mean cen [ 1.         -1.33333333]
pembilang -0.08771929824561414
penyebut 0.6491228070175439
prediksi [2.5] + -0.08771929824561414 / 0.6491228070175439
mean cen (4, 7)
target user 0 5
mean [2.]
sim [0.375 0.   ]
mean cen [-1.33333333  1.        ]
pembilang -0.

,1,2,3,4,5,6,7
1,3.000000,2.864865,2.364865,3.00,2.364865,0.666667,2.000000
2,1.000000,2.000000,4.000000,2.00,2.000000,2.250000,2.333333
3,1.750000,4.000000,1.000000,1.75,2.250000,2.000000,4.000000
4,2.581081,3.250000,2.750000,2.50,3.000000,2.000000,4.000000


## HYBRID

In [34]:
def HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7):
    # Mengonversi ke numpy array
    prediksiUser = np.array(PrediksiUser)
    prediksiItem = np.array(PrediksiItem)
    # Menghitung prediksi hybrid
    rumusHybrid = gamma * prediksiUser + (1 - gamma) * prediksiItem    
    return rumusHybrid

In [35]:
cal_HybridDF = pd.DataFrame(HybridFiltering(cal_HitungAllPrediksiUserRJ, cal_HitungAllPrediksiItemRJ), index=dfToyData.index, columns=dfToyData.columns)
cal_HybridDF

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3.000000,2.983726,2.460393,2.51,2.390965,1.336785,2.000000
user-2,1.000000,2.000000,4.000000,2.00,2.000000,1.609953,3.034953
user-3,2.132879,4.000000,1.000000,2.31,2.529521,2.000000,4.000000
user-4,2.612516,3.482273,2.845125,2.71,3.000000,2.000000,4.000000


In [36]:
cal_HybridFiltering = HybridFiltering(cal_HitungAllPrediksiUserRJ, cal_HitungAllPrediksiItemRJ, gamma=0.7)
cal_HybridFiltering

array([[3.        , 2.98372613, 2.46039279, 2.51      , 2.39096484,
        1.33678474, 2.        ],
       [1.        , 2.        , 4.        , 2.        , 2.        ,
        1.60995268, 3.03495268],
       [2.13287879, 4.        , 1.        , 2.31      , 2.52952055,
        2.        , 4.        ],
       [2.61251641, 3.48227273, 2.84512539, 2.71      , 3.        ,
        2.        , 4.        ]])

## TOPN

In [37]:
def TopNRecommendation(rekomendasiHybrid, N=5):
    topN_per_user = np.argsort(-rekomendasiHybrid, axis=1)[:N]+1
    # tolist
    topN_per_user = topN_per_user.tolist()

    return topN_per_user


In [38]:
cal_TopNRecommendation = TopNRecommendation(cal_HybridFiltering, N=3)
cal_TopNRecommendation

[[1, 2, 4, 3, 5, 7, 6], [3, 7, 2, 4, 5, 6, 1], [2, 7, 5, 4, 1, 6, 3]]

In [39]:
def getTopNRekomendasiV3(prediksiHybrid, ratingMatrix, user_index, N=5):
    pred_user = prediksiHybrid.iloc[user_index].values
    # print("prediksi user", pred_user)
    rating_user = ratingMatrix.iloc[user_index].values
    # print("rating user", rating_user)
    # Item yang belum dirating (nilai 0 atau NaN)
    if np.issubdtype(rating_user.dtype, np.number):
        belum_dirating_idx = np.where(rating_user == 0)[0]
        # print("belum dirating", belum_dirating_idx)
    else:
        belum_dirating_idx = np.where(pd.isna(rating_user))[0]
        print("belum dirating", belum_dirating_idx)
    # Prediksi hanya untuk item yang belum dirating
    pred_belum_dirating = pred_user[belum_dirating_idx]
    # Urutkan dari skor tertinggi
    top_indices = pred_belum_dirating.argsort()[::-1][:N]
    # Ambil ID item-nya (pakai to_numpy() supaya aman)
    top_item_ids = prediksiHybrid.columns.to_numpy()[belum_dirating_idx][top_indices]
    return list(top_item_ids)

In [40]:
cal_TopNRekomendasiV3 = getTopNRekomendasiV3(cal_HybridDF, dfToyData, 0, N=3)
print(f"Top N rekomendasi untuk user-1: {cal_TopNRekomendasiV3}")

Top N rekomendasi untuk user-1: ['item-2', 'item-4', 'item-3']


In [41]:
def getTopNRekomendasiV2(prediksiHybrid, ratingMatrix, user_index, N=5):
    pred_user = prediksiHybrid.iloc[user_index].values
    rating_user = ratingMatrix.iloc[user_index].values
    # Item yang belum dirating (nilai 0 atau NaN)
    if np.issubdtype(rating_user.dtype, np.number):
        belum_dirating_idx = np.where(rating_user == 0)[0]
    else:
        belum_dirating_idx = np.where(pd.isna(rating_user))[0]
    # Prediksi hanya untuk item yang belum dirating
    pred_belum_dirating = pred_user[belum_dirating_idx]
    # Urutkan dari skor tertinggi
    top_indices = pred_belum_dirating.argsort()[::-1][:N]
    # Ambil ID item asli
    top_item_ids = prediksiHybrid.columns.to_numpy()[belum_dirating_idx][top_indices]
    # Convert 'item-2' → 2 (kalau item ID masih dalam bentuk string)
    def parse_item_id(item):
        if isinstance(item, str) and item.startswith("item-"):
            return int(item.split("-")[1])
        return item  # sudah angka
    return [parse_item_id(i) for i in top_item_ids]

In [42]:
cal_TopNRekomendasiV2 = getTopNRekomendasiV2(cal_HybridDF, dfToyData, 0, N=3)
print(f"Top N rekomendasi untuk user-1: {cal_TopNRekomendasiV2}")

Top N rekomendasi untuk user-1: [2, 4, 3]
